# MOSES Treatment Optimization Lab

## Meta-Optimizing Semantic Evolutionary Search for Dermatology

This lab implements the MOSES (Meta-Optimizing Semantic Evolutionary Search) cognitive component for the SkinTwin architecture. MOSES uses evolutionary algorithms to discover optimal treatment patterns for dermatological conditions.

### Key Capabilities

- **Treatment Pattern Discovery**: Evolutionary search through treatment space
- **Clinical Efficacy Optimization**: Multi-objective fitness functions
- **Safety-Constrained Evolution**: Medical safety boundaries
- **Evidence-Based Crossover**: Knowledge-guided genetic operators
- **Pattern Mining**: Extract successful treatment combinations

### Architecture Integration

```
┌─────────────────────────────────────────────────────────────┐
│                    SkinTwin Cognitive Layer                  │
├─────────────────────────────────────────────────────────────┤
│  AtomSpace  │    PLN     │   MOSES    │   ESN    │  ECAN   │
│  Knowledge  │  Reasoning │ Optimization│ Temporal │Attention│
│    Graph    │   Engine   │   Engine   │  Model   │  Alloc  │
├─────────────────────────────────────────────────────────────┤
│                   RegimAI Gateway API                        │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Environment Setup
import os
import json
import random
import numpy as np
from typing import List, Dict, Any, Tuple, Optional
from dataclasses import dataclass, field
from enum import Enum
from copy import deepcopy
import hashlib
from datetime import datetime

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

GATEWAY_URL = os.getenv('REGIMA_GATEWAY_URL', 'http://localhost:3000')
API_KEY = os.getenv('REGIMA_API_KEY', 'demo-key')

## 1. Treatment Domain Model

Define the treatment representation schema for evolutionary optimization.

In [ ]:
class TreatmentType(Enum):
    """Types of dermatological treatments"""
    TOPICAL = "topical"
    ORAL = "oral"
    PROCEDURAL = "procedural"
    LIFESTYLE = "lifestyle"
    DIETARY = "dietary"

class SkinCondition(Enum):
    """Supported dermatological conditions"""
    ACNE_VULGARIS = "acne_vulgaris"
    PSORIASIS = "psoriasis"
    ECZEMA = "eczema"
    ROSACEA = "rosacea"
    MELASMA = "melasma"
    SEBORRHEIC_DERMATITIS = "seborrheic_dermatitis"
    ALOPECIA = "alopecia"
    HYPERPIGMENTATION = "hyperpigmentation"

@dataclass
class TreatmentGene:
    """A single treatment component in the genetic representation"""
    ingredient: str
    concentration: float  # 0.0 to 1.0 normalized
    frequency: str  # "daily", "twice_daily", "weekly", etc.
    treatment_type: TreatmentType
    duration_weeks: int
    
    def to_dict(self) -> Dict:
        return {
            "ingredient": self.ingredient,
            "concentration": self.concentration,
            "frequency": self.frequency,
            "treatment_type": self.treatment_type.value,
            "duration_weeks": self.duration_weeks
        }

@dataclass
class TreatmentChromosome:
    """A complete treatment protocol represented as a chromosome"""
    genes: List[TreatmentGene]
    condition: SkinCondition
    fitness: float = 0.0
    generation: int = 0
    id: str = field(default_factory=lambda: hashlib.md5(str(datetime.now()).encode()).hexdigest()[:8])
    
    def to_dict(self) -> Dict:
        return {
            "id": self.id,
            "condition": self.condition.value,
            "genes": [g.to_dict() for g in self.genes],
            "fitness": self.fitness,
            "generation": self.generation
        }

print("Treatment domain model loaded successfully")

## 2. Dermatology Treatment Knowledge Base

Define the treatment ingredient knowledge for evolutionary search.

In [ ]:
# Treatment ingredient database with clinical properties
TREATMENT_INGREDIENTS = {
    # Acne treatments
    "benzoyl_peroxide": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.ACNE_VULGARIS],
        "efficacy_base": 0.75,
        "safety_score": 0.85,
        "concentration_range": (0.025, 0.10),
        "contraindications": ["sensitive_skin", "rosacea"],
        "synergies": ["adapalene", "clindamycin"],
        "conflicts": ["tretinoin"]
    },
    "salicylic_acid": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.ACNE_VULGARIS, SkinCondition.SEBORRHEIC_DERMATITIS],
        "efficacy_base": 0.70,
        "safety_score": 0.90,
        "concentration_range": (0.005, 0.02),
        "contraindications": ["aspirin_allergy"],
        "synergies": ["niacinamide"],
        "conflicts": []
    },
    "adapalene": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.ACNE_VULGARIS],
        "efficacy_base": 0.80,
        "safety_score": 0.75,
        "concentration_range": (0.001, 0.003),
        "contraindications": ["pregnancy", "sun_sensitivity"],
        "synergies": ["benzoyl_peroxide"],
        "conflicts": ["vitamin_c"]
    },
    "niacinamide": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.ACNE_VULGARIS, SkinCondition.HYPERPIGMENTATION, SkinCondition.ROSACEA],
        "efficacy_base": 0.65,
        "safety_score": 0.95,
        "concentration_range": (0.02, 0.10),
        "contraindications": [],
        "synergies": ["hyaluronic_acid", "zinc"],
        "conflicts": []
    },
    
    # Psoriasis treatments
    "coal_tar": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.PSORIASIS, SkinCondition.SEBORRHEIC_DERMATITIS],
        "efficacy_base": 0.70,
        "safety_score": 0.70,
        "concentration_range": (0.005, 0.05),
        "contraindications": ["photosensitivity"],
        "synergies": ["salicylic_acid"],
        "conflicts": []
    },
    "calcipotriol": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.PSORIASIS],
        "efficacy_base": 0.75,
        "safety_score": 0.80,
        "concentration_range": (0.00005, 0.00005),
        "contraindications": ["hypercalcemia"],
        "synergies": ["betamethasone"],
        "conflicts": []
    },
    
    # Eczema treatments
    "hydrocortisone": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.ECZEMA, SkinCondition.PSORIASIS],
        "efficacy_base": 0.80,
        "safety_score": 0.70,
        "concentration_range": (0.005, 0.025),
        "contraindications": ["skin_infection", "long_term_use"],
        "synergies": [],
        "conflicts": []
    },
    "ceramides": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.ECZEMA],
        "efficacy_base": 0.60,
        "safety_score": 0.99,
        "concentration_range": (0.01, 0.05),
        "contraindications": [],
        "synergies": ["hyaluronic_acid", "cholesterol"],
        "conflicts": []
    },
    
    # Rosacea treatments
    "azelaic_acid": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.ROSACEA, SkinCondition.ACNE_VULGARIS, SkinCondition.HYPERPIGMENTATION],
        "efficacy_base": 0.72,
        "safety_score": 0.88,
        "concentration_range": (0.10, 0.20),
        "contraindications": [],
        "synergies": ["niacinamide"],
        "conflicts": []
    },
    "metronidazole": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.ROSACEA],
        "efficacy_base": 0.75,
        "safety_score": 0.82,
        "concentration_range": (0.0075, 0.01),
        "contraindications": ["alcohol_use"],
        "synergies": [],
        "conflicts": []
    },
    
    # Hyperpigmentation treatments
    "vitamin_c": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.HYPERPIGMENTATION, SkinCondition.MELASMA],
        "efficacy_base": 0.68,
        "safety_score": 0.92,
        "concentration_range": (0.10, 0.20),
        "contraindications": [],
        "synergies": ["vitamin_e", "ferulic_acid"],
        "conflicts": ["adapalene", "retinol"]
    },
    "kojic_acid": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.HYPERPIGMENTATION, SkinCondition.MELASMA],
        "efficacy_base": 0.65,
        "safety_score": 0.80,
        "concentration_range": (0.01, 0.04),
        "contraindications": ["sensitive_skin"],
        "synergies": ["glycolic_acid"],
        "conflicts": []
    },
    "arbutin": {
        "type": TreatmentType.TOPICAL,
        "conditions": [SkinCondition.HYPERPIGMENTATION, SkinCondition.MELASMA],
        "efficacy_base": 0.62,
        "safety_score": 0.90,
        "concentration_range": (0.01, 0.02),
        "contraindications": [],
        "synergies": ["vitamin_c"],
        "conflicts": []
    },
    
    # General skincare
    "hyaluronic_acid": {
        "type": TreatmentType.TOPICAL,
        "conditions": list(SkinCondition),
        "efficacy_base": 0.50,
        "safety_score": 0.99,
        "concentration_range": (0.01, 0.02),
        "contraindications": [],
        "synergies": ["ceramides", "glycerin"],
        "conflicts": []
    },
    "sunscreen_spf50": {
        "type": TreatmentType.TOPICAL,
        "conditions": list(SkinCondition),
        "efficacy_base": 0.40,
        "safety_score": 0.98,
        "concentration_range": (1.0, 1.0),
        "contraindications": [],
        "synergies": [],
        "conflicts": []
    }
}

# Lifestyle interventions
LIFESTYLE_INTERVENTIONS = {
    "stress_management": {
        "type": TreatmentType.LIFESTYLE,
        "conditions": [SkinCondition.PSORIASIS, SkinCondition.ECZEMA, SkinCondition.ACNE_VULGARIS],
        "efficacy_modifier": 0.15
    },
    "sleep_optimization": {
        "type": TreatmentType.LIFESTYLE,
        "conditions": list(SkinCondition),
        "efficacy_modifier": 0.10
    },
    "anti_inflammatory_diet": {
        "type": TreatmentType.DIETARY,
        "conditions": [SkinCondition.PSORIASIS, SkinCondition.ECZEMA, SkinCondition.ACNE_VULGARIS],
        "efficacy_modifier": 0.12
    },
    "dairy_elimination": {
        "type": TreatmentType.DIETARY,
        "conditions": [SkinCondition.ACNE_VULGARIS],
        "efficacy_modifier": 0.08
    }
}

print(f"Loaded {len(TREATMENT_INGREDIENTS)} treatment ingredients")
print(f"Loaded {len(LIFESTYLE_INTERVENTIONS)} lifestyle interventions")

## 3. MOSES Evolutionary Engine

Implementation of the Meta-Optimizing Semantic Evolutionary Search algorithm.

In [ ]:
class MOSESOptimizer:
    """Meta-Optimizing Semantic Evolutionary Search for treatment optimization"""
    
    def __init__(
        self,
        population_size: int = 100,
        max_generations: int = 50,
        tournament_size: int = 7,
        crossover_rate: float = 0.8,
        mutation_rate: float = 0.1,
        elitism_count: int = 5
    ):
        self.population_size = population_size
        self.max_generations = max_generations
        self.tournament_size = tournament_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.elitism_count = elitism_count
        
        self.population: List[TreatmentChromosome] = []
        self.best_individual: Optional[TreatmentChromosome] = None
        self.generation_history: List[Dict] = []
        
        # Fitness weights
        self.fitness_weights = {
            "clinical_efficacy": 0.40,
            "safety_score": 0.30,
            "adherence_score": 0.15,
            "cost_effectiveness": 0.15
        }
    
    def initialize_population(self, condition: SkinCondition, constraints: Dict = None) -> None:
        """Initialize population with random treatment protocols"""
        self.population = []
        constraints = constraints or {}
        
        # Get valid ingredients for this condition
        valid_ingredients = [
            (name, props) for name, props in TREATMENT_INGREDIENTS.items()
            if condition in props["conditions"]
            and name not in constraints.get("avoid_ingredients", [])
        ]
        
        max_products = constraints.get("max_products", 5)
        
        for _ in range(self.population_size):
            num_genes = random.randint(2, min(max_products, len(valid_ingredients)))
            selected = random.sample(valid_ingredients, num_genes)
            
            genes = []
            for name, props in selected:
                conc_range = props["concentration_range"]
                gene = TreatmentGene(
                    ingredient=name,
                    concentration=random.uniform(conc_range[0], conc_range[1]),
                    frequency=random.choice(["daily", "twice_daily", "every_other_day"]),
                    treatment_type=props["type"],
                    duration_weeks=random.randint(4, 12)
                )
                genes.append(gene)
            
            chromosome = TreatmentChromosome(
                genes=genes,
                condition=condition,
                generation=0
            )
            self.population.append(chromosome)
        
        # Evaluate initial fitness
        for individual in self.population:
            individual.fitness = self.evaluate_fitness(individual)
        
        self.population.sort(key=lambda x: x.fitness, reverse=True)
        self.best_individual = self.population[0]
        
        print(f"Initialized population with {len(self.population)} individuals")
        print(f"Best initial fitness: {self.best_individual.fitness:.4f}")
    
    def evaluate_fitness(self, chromosome: TreatmentChromosome) -> float:
        """Evaluate fitness of a treatment protocol"""
        scores = {
            "clinical_efficacy": self._calculate_efficacy(chromosome),
            "safety_score": self._calculate_safety(chromosome),
            "adherence_score": self._calculate_adherence(chromosome),
            "cost_effectiveness": self._calculate_cost_effectiveness(chromosome)
        }
        
        # Weighted sum
        fitness = sum(
            scores[key] * self.fitness_weights[key]
            for key in scores
        )
        
        # Apply synergy bonus
        synergy_bonus = self._calculate_synergy_bonus(chromosome)
        fitness *= (1 + synergy_bonus)
        
        # Apply conflict penalty
        conflict_penalty = self._calculate_conflict_penalty(chromosome)
        fitness *= (1 - conflict_penalty)
        
        return min(1.0, max(0.0, fitness))
    
    def _calculate_efficacy(self, chromosome: TreatmentChromosome) -> float:
        """Calculate clinical efficacy score"""
        if not chromosome.genes:
            return 0.0
        
        total_efficacy = 0.0
        for gene in chromosome.genes:
            props = TREATMENT_INGREDIENTS.get(gene.ingredient, {})
            base_efficacy = props.get("efficacy_base", 0.5)
            
            # Adjust for concentration (closer to optimal is better)
            conc_range = props.get("concentration_range", (0, 1))
            optimal_conc = (conc_range[0] + conc_range[1]) / 2
            conc_factor = 1 - abs(gene.concentration - optimal_conc) / max(conc_range[1], 0.01)
            
            # Adjust for frequency
            freq_factors = {"daily": 1.0, "twice_daily": 1.1, "every_other_day": 0.8}
            freq_factor = freq_factors.get(gene.frequency, 1.0)
            
            total_efficacy += base_efficacy * conc_factor * freq_factor
        
        return min(1.0, total_efficacy / len(chromosome.genes))
    
    def _calculate_safety(self, chromosome: TreatmentChromosome) -> float:
        """Calculate safety score"""
        if not chromosome.genes:
            return 1.0
        
        safety_scores = []
        for gene in chromosome.genes:
            props = TREATMENT_INGREDIENTS.get(gene.ingredient, {})
            safety_scores.append(props.get("safety_score", 0.5))
        
        # Minimum safety determines overall safety
        return min(safety_scores) * 0.7 + (sum(safety_scores) / len(safety_scores)) * 0.3
    
    def _calculate_adherence(self, chromosome: TreatmentChromosome) -> float:
        """Calculate patient adherence likelihood"""
        if not chromosome.genes:
            return 1.0
        
        # Fewer products = better adherence
        product_penalty = len(chromosome.genes) * 0.05
        
        # Simpler frequencies = better adherence
        freq_scores = {"daily": 1.0, "twice_daily": 0.85, "every_other_day": 0.9}
        avg_freq = sum(freq_scores.get(g.frequency, 0.8) for g in chromosome.genes) / len(chromosome.genes)
        
        return max(0.3, min(1.0, avg_freq - product_penalty))
    
    def _calculate_cost_effectiveness(self, chromosome: TreatmentChromosome) -> float:
        """Calculate cost effectiveness (simplified model)"""
        # More ingredients = higher cost
        base_cost = 0.9 - (len(chromosome.genes) * 0.1)
        return max(0.3, base_cost)
    
    def _calculate_synergy_bonus(self, chromosome: TreatmentChromosome) -> float:
        """Calculate bonus for synergistic ingredient combinations"""
        bonus = 0.0
        ingredients = [g.ingredient for g in chromosome.genes]
        
        for gene in chromosome.genes:
            props = TREATMENT_INGREDIENTS.get(gene.ingredient, {})
            synergies = props.get("synergies", [])
            for syn in synergies:
                if syn in ingredients:
                    bonus += 0.05
        
        return min(0.3, bonus)
    
    def _calculate_conflict_penalty(self, chromosome: TreatmentChromosome) -> float:
        """Calculate penalty for conflicting ingredient combinations"""
        penalty = 0.0
        ingredients = [g.ingredient for g in chromosome.genes]
        
        for gene in chromosome.genes:
            props = TREATMENT_INGREDIENTS.get(gene.ingredient, {})
            conflicts = props.get("conflicts", [])
            for conf in conflicts:
                if conf in ingredients:
                    penalty += 0.15
        
        return min(0.5, penalty)
    
    def tournament_selection(self) -> TreatmentChromosome:
        """Select individual using tournament selection"""
        tournament = random.sample(self.population, min(self.tournament_size, len(self.population)))
        return max(tournament, key=lambda x: x.fitness)
    
    def crossover(self, parent1: TreatmentChromosome, parent2: TreatmentChromosome) -> Tuple[TreatmentChromosome, TreatmentChromosome]:
        """Perform crossover between two parents"""
        if random.random() > self.crossover_rate:
            return deepcopy(parent1), deepcopy(parent2)
        
        # Single-point crossover on genes
        genes1 = parent1.genes.copy()
        genes2 = parent2.genes.copy()
        
        if len(genes1) > 1 and len(genes2) > 1:
            point1 = random.randint(1, len(genes1) - 1)
            point2 = random.randint(1, len(genes2) - 1)
            
            new_genes1 = genes1[:point1] + genes2[point2:]
            new_genes2 = genes2[:point2] + genes1[point1:]
        else:
            new_genes1, new_genes2 = genes1, genes2
        
        child1 = TreatmentChromosome(
            genes=new_genes1,
            condition=parent1.condition,
            generation=parent1.generation + 1
        )
        child2 = TreatmentChromosome(
            genes=new_genes2,
            condition=parent2.condition,
            generation=parent2.generation + 1
        )
        
        return child1, child2
    
    def mutate(self, chromosome: TreatmentChromosome) -> TreatmentChromosome:
        """Apply mutation to a chromosome"""
        if random.random() > self.mutation_rate:
            return chromosome
        
        mutated = deepcopy(chromosome)
        mutation_type = random.choice(["concentration", "frequency", "add", "remove", "swap"])
        
        if mutation_type == "concentration" and mutated.genes:
            gene = random.choice(mutated.genes)
            props = TREATMENT_INGREDIENTS.get(gene.ingredient, {})
            conc_range = props.get("concentration_range", (0, 1))
            gene.concentration = random.uniform(conc_range[0], conc_range[1])
        
        elif mutation_type == "frequency" and mutated.genes:
            gene = random.choice(mutated.genes)
            gene.frequency = random.choice(["daily", "twice_daily", "every_other_day"])
        
        elif mutation_type == "add":
            valid = [
                (n, p) for n, p in TREATMENT_INGREDIENTS.items()
                if mutated.condition in p["conditions"]
                and n not in [g.ingredient for g in mutated.genes]
            ]
            if valid and len(mutated.genes) < 6:
                name, props = random.choice(valid)
                conc_range = props["concentration_range"]
                new_gene = TreatmentGene(
                    ingredient=name,
                    concentration=random.uniform(conc_range[0], conc_range[1]),
                    frequency=random.choice(["daily", "twice_daily"]),
                    treatment_type=props["type"],
                    duration_weeks=random.randint(4, 12)
                )
                mutated.genes.append(new_gene)
        
        elif mutation_type == "remove" and len(mutated.genes) > 1:
            mutated.genes.remove(random.choice(mutated.genes))
        
        elif mutation_type == "swap" and len(mutated.genes) > 1:
            idx = random.randint(0, len(mutated.genes) - 1)
            valid = [
                (n, p) for n, p in TREATMENT_INGREDIENTS.items()
                if mutated.condition in p["conditions"]
                and n not in [g.ingredient for g in mutated.genes]
            ]
            if valid:
                name, props = random.choice(valid)
                conc_range = props["concentration_range"]
                mutated.genes[idx] = TreatmentGene(
                    ingredient=name,
                    concentration=random.uniform(conc_range[0], conc_range[1]),
                    frequency=mutated.genes[idx].frequency,
                    treatment_type=props["type"],
                    duration_weeks=mutated.genes[idx].duration_weeks
                )
        
        return mutated
    
    def evolve_generation(self) -> Dict:
        """Evolve one generation"""
        new_population = []
        
        # Elitism: keep best individuals
        elites = sorted(self.population, key=lambda x: x.fitness, reverse=True)[:self.elitism_count]
        new_population.extend(deepcopy(elites))
        
        # Generate rest of population
        while len(new_population) < self.population_size:
            parent1 = self.tournament_selection()
            parent2 = self.tournament_selection()
            
            child1, child2 = self.crossover(parent1, parent2)
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            
            child1.fitness = self.evaluate_fitness(child1)
            child2.fitness = self.evaluate_fitness(child2)
            
            new_population.extend([child1, child2])
        
        self.population = new_population[:self.population_size]
        self.population.sort(key=lambda x: x.fitness, reverse=True)
        
        if self.population[0].fitness > self.best_individual.fitness:
            self.best_individual = deepcopy(self.population[0])
        
        # Record generation stats
        stats = {
            "generation": len(self.generation_history),
            "best_fitness": self.population[0].fitness,
            "avg_fitness": sum(i.fitness for i in self.population) / len(self.population),
            "worst_fitness": self.population[-1].fitness,
            "diversity": len(set(str(i.genes) for i in self.population)) / len(self.population)
        }
        self.generation_history.append(stats)
        
        return stats
    
    def optimize(
        self,
        condition: SkinCondition,
        constraints: Dict = None,
        verbose: bool = True
    ) -> TreatmentChromosome:
        """Run full optimization"""
        self.initialize_population(condition, constraints)
        
        for gen in range(self.max_generations):
            stats = self.evolve_generation()
            
            if verbose and gen % 10 == 0:
                print(f"Generation {gen}: Best={stats['best_fitness']:.4f}, "
                      f"Avg={stats['avg_fitness']:.4f}, Diversity={stats['diversity']:.2f}")
            
            # Early stopping if converged
            if stats['diversity'] < 0.1:
                if verbose:
                    print(f"Converged at generation {gen}")
                break
        
        return self.best_individual

print("MOSES Optimizer loaded successfully")

## 4. Run Treatment Optimization

Demonstrate MOSES optimization for various skin conditions.

In [ ]:
# Initialize MOSES optimizer
moses = MOSESOptimizer(
    population_size=100,
    max_generations=50,
    tournament_size=7,
    crossover_rate=0.8,
    mutation_rate=0.15
)

# Optimize treatment for acne
print("=" * 60)
print("Optimizing treatment for ACNE VULGARIS")
print("=" * 60)

constraints = {
    "avoid_ingredients": ["hydrocortisone"],  # Avoid steroids for acne
    "max_products": 4
}

best_acne_treatment = moses.optimize(
    condition=SkinCondition.ACNE_VULGARIS,
    constraints=constraints,
    verbose=True
)

print("\n" + "=" * 60)
print("OPTIMAL TREATMENT PROTOCOL")
print("=" * 60)
print(f"Fitness Score: {best_acne_treatment.fitness:.4f}")
print(f"\nRecommended Treatment:")
for i, gene in enumerate(best_acne_treatment.genes, 1):
    print(f"  {i}. {gene.ingredient.replace('_', ' ').title()}")
    print(f"     - Concentration: {gene.concentration*100:.2f}%")
    print(f"     - Frequency: {gene.frequency.replace('_', ' ')}")
    print(f"     - Duration: {gene.duration_weeks} weeks")

In [ ]:
# Optimize treatment for psoriasis
print("\n" + "=" * 60)
print("Optimizing treatment for PSORIASIS")
print("=" * 60)

psoriasis_constraints = {
    "max_products": 3,
    "requires_medical_supervision": True
}

moses_psoriasis = MOSESOptimizer(
    population_size=80,
    max_generations=40
)

best_psoriasis_treatment = moses_psoriasis.optimize(
    condition=SkinCondition.PSORIASIS,
    constraints=psoriasis_constraints,
    verbose=True
)

print("\n" + "=" * 60)
print("OPTIMAL TREATMENT PROTOCOL")
print("=" * 60)
print(f"Fitness Score: {best_psoriasis_treatment.fitness:.4f}")
print(f"\nRecommended Treatment:")
for i, gene in enumerate(best_psoriasis_treatment.genes, 1):
    print(f"  {i}. {gene.ingredient.replace('_', ' ').title()}")
    print(f"     - Concentration: {gene.concentration*100:.2f}%")
    print(f"     - Frequency: {gene.frequency.replace('_', ' ')}")

## 5. Pattern Mining

Extract common successful treatment patterns from the evolved population.

In [ ]:
class PatternMiner:
    """Extract successful treatment patterns from MOSES populations"""
    
    def __init__(self, min_support: float = 0.1):
        self.min_support = min_support
        self.patterns: List[Dict] = []
    
    def mine_patterns(self, population: List[TreatmentChromosome], top_percentile: float = 0.2) -> List[Dict]:
        """Mine patterns from top performing individuals"""
        # Select top performers
        sorted_pop = sorted(population, key=lambda x: x.fitness, reverse=True)
        top_count = max(1, int(len(sorted_pop) * top_percentile))
        top_performers = sorted_pop[:top_count]
        
        # Count ingredient occurrences
        ingredient_counts = {}
        pair_counts = {}
        
        for individual in top_performers:
            ingredients = [g.ingredient for g in individual.genes]
            
            # Single ingredients
            for ing in ingredients:
                ingredient_counts[ing] = ingredient_counts.get(ing, 0) + 1
            
            # Ingredient pairs
            for i, ing1 in enumerate(ingredients):
                for ing2 in ingredients[i+1:]:
                    pair = tuple(sorted([ing1, ing2]))
                    pair_counts[pair] = pair_counts.get(pair, 0) + 1
        
        # Extract patterns above minimum support
        patterns = []
        
        # Single ingredient patterns
        for ing, count in ingredient_counts.items():
            support = count / len(top_performers)
            if support >= self.min_support:
                patterns.append({
                    "type": "single_ingredient",
                    "pattern": [ing],
                    "support": support,
                    "confidence": count / len(population)
                })
        
        # Pair patterns
        for pair, count in pair_counts.items():
            support = count / len(top_performers)
            if support >= self.min_support:
                patterns.append({
                    "type": "ingredient_pair",
                    "pattern": list(pair),
                    "support": support,
                    "confidence": count / len(population)
                })
        
        self.patterns = sorted(patterns, key=lambda x: x["support"], reverse=True)
        return self.patterns
    
    def get_top_patterns(self, n: int = 10) -> List[Dict]:
        """Get top N patterns by support"""
        return self.patterns[:n]

# Mine patterns from acne optimization
miner = PatternMiner(min_support=0.15)
patterns = miner.mine_patterns(moses.population)

print("=" * 60)
print("DISCOVERED TREATMENT PATTERNS (Acne)")
print("=" * 60)

for i, pattern in enumerate(miner.get_top_patterns(8), 1):
    ingredients = " + ".join([p.replace("_", " ").title() for p in pattern["pattern"]])
    print(f"{i}. {ingredients}")
    print(f"   Support: {pattern['support']*100:.1f}% | Type: {pattern['type']}")

## 6. Gateway Integration

Integrate MOSES optimization with the RegimAI Gateway.

In [ ]:
import requests

class MOSESGatewayClient:
    """Client for MOSES optimization through RegimAI Gateway"""
    
    def __init__(self, gateway_url: str, api_key: str):
        self.gateway_url = gateway_url.rstrip('/')
        self.api_key = api_key
        self.headers = {
            "X-API-Key": api_key,
            "Content-Type": "application/json"
        }
    
    def optimize_treatment(
        self,
        condition: str,
        patient_profile: Dict = None,
        constraints: Dict = None,
        optimization_goals: List[str] = None
    ) -> Dict:
        """Request treatment optimization from gateway"""
        payload = {
            "condition": condition,
            "patient_profile": patient_profile or {},
            "constraints": constraints or {},
            "optimization_goals": optimization_goals or ["maximize_efficacy", "minimize_side_effects"]
        }
        
        response = requests.post(
            f"{self.gateway_url}/cognitive/moses/optimize",
            headers=self.headers,
            json=payload
        )
        
        return response.json()
    
    def evaluate_protocol(
        self,
        condition: str,
        treatment_protocol: List[Dict]
    ) -> Dict:
        """Evaluate a treatment protocol's fitness"""
        payload = {
            "condition": condition,
            "protocol": treatment_protocol
        }
        
        response = requests.post(
            f"{self.gateway_url}/cognitive/moses/evaluate",
            headers=self.headers,
            json=payload
        )
        
        return response.json()
    
    def get_patterns(
        self,
        condition: str,
        min_support: float = 0.1
    ) -> Dict:
        """Get discovered treatment patterns for a condition"""
        payload = {
            "condition": condition,
            "min_support": min_support
        }
        
        response = requests.post(
            f"{self.gateway_url}/cognitive/moses/patterns/mine",
            headers=self.headers,
            json=payload
        )
        
        return response.json()

# Example usage (when gateway is running)
# client = MOSESGatewayClient(GATEWAY_URL, API_KEY)
# result = client.optimize_treatment(
#     condition="acne_vulgaris",
#     patient_profile={"age": 22, "skin_type": "oily"},
#     constraints={"max_products": 3}
# )

print("MOSES Gateway Client ready")
print(f"Gateway URL: {GATEWAY_URL}")

## 7. Visualization

Visualize the optimization process and results.

In [ ]:
import matplotlib.pyplot as plt

def plot_evolution_history(history: List[Dict], title: str = "MOSES Evolution"):
    """Plot the evolution history"""
    generations = [h["generation"] for h in history]
    best_fitness = [h["best_fitness"] for h in history]
    avg_fitness = [h["avg_fitness"] for h in history]
    diversity = [h["diversity"] for h in history]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    
    # Fitness plot
    ax1.plot(generations, best_fitness, 'b-', label='Best Fitness', linewidth=2)
    ax1.plot(generations, avg_fitness, 'g--', label='Average Fitness', linewidth=1.5)
    ax1.fill_between(generations, avg_fitness, best_fitness, alpha=0.2)
    ax1.set_xlabel('Generation')
    ax1.set_ylabel('Fitness Score')
    ax1.set_title(f'{title} - Fitness Evolution')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Diversity plot
    ax2.plot(generations, diversity, 'r-', label='Population Diversity', linewidth=2)
    ax2.set_xlabel('Generation')
    ax2.set_ylabel('Diversity')
    ax2.set_title('Population Diversity Over Time')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot evolution for acne optimization
if moses.generation_history:
    plot_evolution_history(moses.generation_history, "Acne Treatment Optimization")

## 8. Export Results

Export optimization results for integration with AtomSpace and PLN.

In [ ]:
def export_to_atomspace_format(chromosome: TreatmentChromosome) -> Dict:
    """Convert treatment chromosome to AtomSpace representation"""
    atoms = []
    
    # Create treatment protocol node
    protocol_atom = {
        "type": "ConceptNode",
        "name": f"TreatmentProtocol:{chromosome.id}",
        "truth_value": {
            "strength": chromosome.fitness,
            "confidence": 0.9
        }
    }
    atoms.append(protocol_atom)
    
    # Create ingredient nodes and links
    for gene in chromosome.genes:
        ingredient_atom = {
            "type": "ConceptNode",
            "name": f"Ingredient:{gene.ingredient}"
        }
        atoms.append(ingredient_atom)
        
        # Link ingredient to protocol
        link_atom = {
            "type": "EvaluationLink",
            "predicate": "contains_ingredient",
            "arguments": [
                f"TreatmentProtocol:{chromosome.id}",
                f"Ingredient:{gene.ingredient}"
            ],
            "truth_value": {
                "strength": gene.concentration,
                "confidence": 0.95
            }
        }
        atoms.append(link_atom)
    
    # Create condition treatment link
    treats_link = {
        "type": "EvaluationLink",
        "predicate": "treats_condition",
        "arguments": [
            f"TreatmentProtocol:{chromosome.id}",
            f"Condition:{chromosome.condition.value}"
        ],
        "truth_value": {
            "strength": chromosome.fitness,
            "confidence": 0.85
        }
    }
    atoms.append(treats_link)
    
    return {
        "protocol_id": chromosome.id,
        "condition": chromosome.condition.value,
        "fitness": chromosome.fitness,
        "atoms": atoms
    }

# Export best treatment
atomspace_export = export_to_atomspace_format(best_acne_treatment)

print("=" * 60)
print("ATOMSPACE EXPORT FORMAT")
print("=" * 60)
print(json.dumps(atomspace_export, indent=2))

## Summary

This lab demonstrates:

1. **MOSES Evolutionary Search**: Meta-optimizing treatment discovery using genetic algorithms
2. **Treatment Domain Model**: Representation of dermatological treatments as genetic chromosomes
3. **Multi-Objective Fitness**: Balancing efficacy, safety, adherence, and cost
4. **Pattern Mining**: Extracting successful treatment combinations
5. **AtomSpace Integration**: Exporting results for knowledge graph storage

### Next Steps

- Integrate with PLN for evidence-based treatment reasoning
- Connect to ESN for temporal progression modeling
- Deploy as RegimAI Gateway cognitive endpoint
- Add real clinical trial data for validation